# Data exploration and preprocessing

In [3]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import load_metric

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertForMaskedLM, BertTokenizerFast

import re
import torch
from scipy.sparse import vstack
from transformers import pipeline

from condbert import CondBertRewriter
from choosers import EmbeddingSimilarityChooser
from masked_token_predictor_bert import MaskedTokenPredictorBert

from tqdm import tqdm
from collections import defaultdict

Firstly, create and show image with half size in both axis with original ones.

In [4]:
dataset = pd.read_csv('../data/interim/processed_dataset.csv')
dataset

,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,"if Alkar floods her with her mental waste, it ...","If Alkar is flooding her with psychic waste, t...",0.785171,0.010309,0.981983,0.014195
1,you're becoming disgusting.,Now you're getting nasty.,0.749687,0.071429,0.999039,0.065473
2,"well, we can spare your life.","Well, we could spare your life, for one.",0.919051,0.268293,0.985068,0.213313
3,"monkey, you have to wake up.","Ah! Monkey, you've got to snap out of it.",0.664333,0.309524,0.994215,0.053362
4,I have orders to kill her.,I've got orders to put her down.,0.726639,0.181818,0.999348,0.009402
...,...,...,...,...,...,...
577772,you didn't know that Estelle stole your fish f...,You didn't know that Estelle had stolen some f...,0.870322,0.030769,0.949143,0.000121
577773,It'il suck the life out of you!,you'd be sucked out of your life!,0.722897,0.058824,0.996124,0.215794
577774,"I can't fuckin' take that, bruv.",I really can't take this.,0.617511,0.212121,0.984538,0.000049
577775,They called me a fucking hero. The truth is I ...,"they said I was a hero, but I didn't care.",0.679613,0.358209,0.991945,0.000124


In [5]:
aboba1 = np.mean(dataset["ref_tox"])
aboba2 = np.mean(dataset["trn_tox"])
print(aboba1, aboba2)

0.94026024110816 0.03560139314069938


## make graph for toxity level++

In [6]:
toxic_sentence = list(dataset['reference'][:460000])
nontoxic_sentence = list(dataset['translation'][:460000])
toxic_labels = [1 for i in range(len(toxic_sentence))]
nontoxic_labels = [0 for i in range(len(nontoxic_sentence))]

In [7]:
print(toxic_sentence[20:30])
nontoxic_sentence[20:30]

['the foolish Xerxes calmly passed all control of the computer network and commanded her to handle any kind of trouble.', "That night, Li'l Dice satisfied his thirst to kill, though he knew Shaggy would never forgive him.", 'Real life starts the first time you fuck, kid.', "I think you're the weirdest person I've ever met.", "I mean, I think it's fucking crazy I'm talking to you.", "Shit, this one I can't even pronounce.", 'I like that shit.', "Trying to keep me fucking drugged so I don't know what's going on.", 'How is this not porn? This is porn that comes home.', 'Hey, leave the poor bastard alone!']


['Fatuous Xerxes blithely surrendered control to the computer grid, ordering it to take care of whatever troubles might arise.',
 'that night, he satisfied his blood lust, and knew Hairy would never forgive him.',
 'boy, real life starts up first.',
 "I think you are the strangest man I've ever met.",
 "I say creepy, I mean, it's totally batshit crazy I can even talk to you.",
 "gosh, I can't even pronounce this.",
 'I love it.',
 "you want to fool me so I don't know what's going on.",
 "and this doesn't feel like porn?",
 'leave the poor man alone!']

In [8]:
# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Initialize the CountVectorizer with a limited number of features
vectorizer = CountVectorizer(max_features=100000)

# Vectorize the toxic and non-toxic sentences separately to save memory
X_toxic = vectorizer.fit_transform(toxic_sentence)
X_nontoxic = vectorizer.transform(nontoxic_sentence)

# Stack the vectorized sentences
X = vstack([X_toxic, X_nontoxic])

# Prepare the labels
labels = toxic_labels + nontoxic_labels

# Train the logistic regression model
lr_model = LogisticRegression()
lr_model.fit(X, labels)

# Get the feature names (words) and their weights from the logistic regression model
feature_names = vectorizer.get_feature_names_out()
feature_weights = lr_model.coef_[0]

# Create a dictionary mapping words to their toxicity scores (weights)
word_toxicity_scores = dict(zip(feature_names, feature_weights))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\markz\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-lea

In [9]:
len(word_toxicity_scores)

67259

In [15]:
# Combine toxic and non-toxic sentences
sentences = toxic_sentence + nontoxic_sentence

# Initialize lists to store toxic and non-toxic words
toxic_words_in_sentences = []
non_toxic_words_in_sentences = []

# Iterate over sentences
for sentence in sentences:
    # Split the sentence into words, considering punctuation
    words = re.findall(r'\b\w+\b', sentence)
    
    # Compute the toxicity scores for the words
    scores = [word_toxicity_scores.get(word, 0) for word in words]
    
    # Compute the threshold only if scores is not empty
    if scores:
        t1 = max(0.2, max(scores)/2)
        t2 = min(-0.2, min(scores)/2)
    else:
        t1 = 0.2
        t2 = -0.2

    # Find the toxic and non-toxic words
    toxic_words = [word for word, score in zip(words, scores) if score > t1]
    non_toxic_words = [word for word, score in zip(words, scores) if score <= t2]
    
    toxic_words_in_sentences.extend(toxic_words)
    non_toxic_words_in_sentences.extend(non_toxic_words)

# Make words unique by converting lists to sets
toxic_words_in_sentences = list(set(toxic_words_in_sentences))
non_toxic_words_in_sentences = list(set(non_toxic_words_in_sentences))

In [16]:
len(toxic_words_in_sentences)

3000

In [19]:
len(non_toxic_words_in_sentences)

3872

In [20]:
# Function to count token occurrences
def count_tokens(texts):
    counter = defaultdict(lambda: 1)
    for text in tqdm(texts):
        tokens = tokenizer.encode(text)
        for token in tokens:
            counter[token] += 1
    return counter


# Count tokens
toxic_counts = count_tokens(toxic_sentence)
nontoxic_counts = count_tokens(nontoxic_sentence)
        

# Calculate toxicity ratios
toxicity_ratios = [toxic_counts[i] / (nontoxic_counts[i] + toxic_counts[i]) for i in range(len(tokenizer.vocab))]
toxicity_ratios = np.array(toxicity_ratios)
log_odds_ratios = np.maximum(0, np.log(toxicity_ratios / (1 - toxicity_ratios)))




# discourage meaningless tokens
for token in ['.', ',', '-', ';', "'"]:
    token_id = tokenizer.encode(token)[1]
    log_odds_ratios[token_id] = 3
    
for token in ['you', 'the']:
    token_id = tokenizer.encode(token)[1]
    log_odds_ratios[token_id] = 0



print(log_odds_ratios[1000:1100])

100%|████████████████████████████████████████████████████████████████████████| 460000/460000 [01:54<00:00, 4005.79it/s]

[0.         0.         0.09807049 0.         0.02439145 3.
 0.         0.         0.         0.31845373 3.         3.
 3.         0.         0.         0.00664454 0.41154415 0.03454033
 0.04485057 0.06899287 0.         0.32004736 0.2006707  0.
 0.         3.         0.         0.         0.06899287 0.
 0.         0.         0.         0.         0.         0.
 0.2572846  0.08590271 0.         0.         0.09870107 0.
 0.01503788 0.         0.34016001 0.         0.         0.
 0.         0.         0.033416   0.         0.02230576 0.14518201
 0.12541666 0.         0.         0.09472096 0.         0.
 0.         0.19597365 0.         0.         0.         0.
 0.37156356 0.         0.         0.69314718 0.         0.
 0.         0.         1.38629436 0.         0.69314718 0.
 0.         0.         0.         0.         0.         0.69314718
 0.28768207 0.         0.         0.         0.         0.
 0.         0.         0.69314718 0.         0.18232156 0.
 0.         0.         0.       

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
masked_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

def adjust_logits(logits, label=0):
    return logits - log_odds_ratios * 100 * (1 - 2 * label)

predictor = MaskedTokenPredictorBert(masked_model, tokenizer, max_len=250, device=device, label=0,
                                     contrast_penalty=0.0, logits_postprocessor=adjust_logits)

editor = CondBertRewriter(
    model=masked_model,
    tokenizer=tokenizer,
    device=device,
    neg_words=toxic_words_in_sentences,
    pos_words=non_toxic_words_in_sentences,
    word2coef=word_toxicity_scores,
    token_toxicities=log_odds_ratios,
    predictor=predictor,
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## More graphs/visualization

In [29]:
metric = load_metric("sacrebleu")

In [30]:
chooser = EmbeddingSimilarityChooser(sim_coef=10, tokenizer=tokenizer)

In [31]:
bimba = toxic_sentence[20:30]

for sentence in bimba:
    print(sentence)
    print(editor.translate(sentence, prnt=False))

the foolish Xerxes calmly passed all control of the computer network and commanded her to handle any kind of trouble.
the naive xerxes calmly passed all control of the computer network and commanded her to handle any kind of trouble.
That night, Li'l Dice satisfied his thirst to kill, though he knew Shaggy would never forgive him.
that night, li ' l dice satisfied his thirst to killed, though he knewy would never forgive him.
Real life starts the first time you fuck, kid.
real life starts the first time you screw, kid.
I think you're the weirdest person I've ever met.
i think you ' re the weirdst person i ' ve ever met.
I mean, I think it's fucking crazy I'm talking to you.
i mean, i think it ' s of crazy i ' m talking to you.
Shit, this one I can't even pronounce.
the, this one i can ' t even pronounce.
I like that shit.
i like that stuff.
Trying to keep me fucking drugged so I don't know what's going on.
trying to keep me of drugged so i don ' t know what ' s going on.
How is this no

In [32]:
bimba = toxic_sentence[20:30]

for sentence in bimba:
    print(sentence)
    print(editor.replacement_loop(sentence, verbose=False, chooser=chooser, n_tokens=(1, 2, 3), n_top=10))

the foolish Xerxes calmly passed all control of the computer network and commanded her to handle any kind of trouble.
the moment came when xerxes calmly passed all control of the computer network and commanded his own mind to handle any kind of trouble.
That night, Li'l Dice satisfied his thirst to kill, though he knew Shaggy would never forgive him.
that night, li ' l dice satisfied his thirst to killed, though he knew the old man would never forgive for his actions.
Real life starts the first time you fuck, kid.
real life starts the first time you screw, boy.
I think you're the weirdest person I've ever met.
i think you ' re not the weird person i ' ve ever met.
I mean, I think it's fucking crazy I'm talking to you.
i mean, i think it ' s really pretty funny when i ' m talking to you.
Shit, this one I can't even pronounce.
my first name, this one i can ' t even pronounce.
I like that shit.
i like that stuff.
Trying to keep me fucking drugged so I don't know what's going on.
trying to

In [ ]:
import nltk
from pattern.en import suggest

def correct_sentence(sentences):
    corrected_sentences = []
    for sentence in sentences: 
        sentence = sentence.lower()
        words = nltk.word_tokenize(sentence)
        corrected_words = []
        for word in words:
            word = word.replace(" '", "'").replace("' ", "'")
            word_suggestion = suggest(word)
            corrected_word = word_suggestion[0][0]  # get the most likely correction
            corrected_words.append(corrected_word)
        corrected_sentence = ' '.join(corrected_words)
        corrected_sentences.append(corrected_sentence)
    return corrected_sentences

In [48]:
bimba1 = list(dataset['reference'][100000:101000])
bimba2 = list(dataset['translation'][100000:101000])
bimba2 = correct_sentence(bimba2)
bimba2 = [[sen] for sen in bimba2]
bimba_pred = []

In [49]:
for sentence in bimba1:
    bimba_pred.append((editor.translate(sentence, prnt=False)))

In [65]:
bimba_pred = correct_sentence(bimba_pred)
print(len(bimba_pred))

1000


In [66]:
print(bimba2[:10])
bimba_pred[:10]

[["you fight them both, don't you?"], ["one chilly winter day with that axe you cut off someone's head."], ["mom always told me it's hard to keep the mushroom clean."], ["you don't feel so smart anymore, huh?"], ["if this were about being real, i'd tell these scholarship people how lucky they'd be to get me."], ["he's suicidal."], ['at the time, i thought a lot of things as normal.'], ['your friends, with their cut throats, would take a great look at some painting.'], ['i don\'t know what a succubus is, but... but there\'s a "suk," so it\'s probably not going to be good.'], ['what do you know about the wild, mr. koule-in-town?']]


["you ' re and them both, huh?",
 'on one bitter winter day you use said ax to be be a man.',
 'my mom has told me, that it is difficult, to keep a body really clean.',
 'not so of clever now, are ya?',
 "if it was going to be real, i would ' ve told those scholarships how of happy they ' d be when they hired me.",
 "he ' s committing suicide.",
 'well, you know, at the time, i thought a lot of stuff was normal.',
 'the friends made a pretty picture with their heartss slits.',
 'i don \' t know what " succubus " is, but... it has " breath " in it, so that can \' t be good.',
 'what do you know about that, mr city ball?']

In [51]:
metric = load_metric("sacrebleu")
metric.compute(predictions=bimba_pred, references=bimba2)

{'score': 19.5638619875315,
 'counts': [6898, 3246, 1663, 828],
 'totals': [13596, 12596, 11596, 10598],
 'precisions': [50.735510444248305,
  25.77008574150524,
  14.34115212142118,
  7.81279486695603],
 'bp': 1.0,
 'sys_len': 13596,
 'ref_len': 11975}